<a href="https://colab.research.google.com/github/sehapark1/sehapark1.github.io/blob/projects/Location%20Tracking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from geopy.distance import geodesic
from datetime import datetime

# Load fast food locations
fast_food_df = pd.read_csv('/content/fast_food_locations.csv')
# Assuming the columns are 'Place', 'Latitude', 'Longitude'
fast_food_locations = fast_food_df[['Place', 'Latitude', 'Longitude']]

# Load professor location data
transformed_file_path = '/content/drive/MyDrive/Colab Notebooks/Transformed_LocationData.pkl'
professor_locations = pd.read_pickle(transformed_file_path)

# Reset index to make 'timestamp' a column
professor_locations.reset_index(inplace=True)

# Convert 'timestamp' to datetime and remove timezone if it exists
professor_locations['timestamp'] = pd.to_datetime(professor_locations['timestamp']).dt.tz_localize(None)

# Define a naive start date for filtering
start_date = datetime(2024, 1, 1)

# Filter professor locations to only include dates from 2024-01-01 onwards
professor_locations = professor_locations[professor_locations['timestamp'] >= start_date]

# Add a date column to group by date
professor_locations['date'] = professor_locations['timestamp'].dt.date

# Keep only unique daily locations
daily_locations = professor_locations.drop_duplicates(subset=['date', 'latitude', 'longitude'])

# Initialize dictionary to store counts of visits for each fast food location
visit_counts = {Place: 0 for Place in fast_food_df['Place']}
plot_data = []

# Initialize a set to track daily visits to each restaurant
daily_visit_tracker = {}

# Calculate visits by finding the closest fast food location for each daily unique location
for _, prof_row in daily_locations.iterrows():
    prof_location = (prof_row['latitude'], prof_row['longitude'])
    visit_date = prof_row['date']
    visited_today = False  # Track if this professor's location resulted in a visit today

    for _, food_row in fast_food_df.iterrows():
        food_location = (food_row['Latitude'], food_row['Longitude'])

        # Check if the professor's location matches a fast food location
        if (round(prof_location[0], 4), round(prof_location[1], 4)) == (round(food_location[0], 4), round(food_location[1], 4)):
            # Only count one visit per day for each restaurant
            if (food_row['Place'], visit_date) not in daily_visit_tracker:
                visit_counts[food_row['Place']] += 1
                daily_visit_tracker[(food_row['Place'], visit_date)] = True
                visited_today = True

                # Add data to plot
                plot_data.append({
                    'Type': 'Professor',
                    'Name': 'Professor',
                    'Latitude': prof_location[0],
                    'Longitude': prof_location[1],
                    'Visit Count': None
                })
                plot_data.append({
                    'Type': 'Restaurant',
                    'Name': food_row['Place'],
                    'Latitude': food_location[0],
                    'Longitude': food_location[1],
                    'Visit Count': visit_counts[food_row['Place']]
                })

            # Stop checking other restaurants if a visit was already recorded
            if visited_today:
                break

# Convert plot data to DataFrame
plot_data_df = pd.DataFrame(plot_data)


In [ ]:
import plotly.express as px
#Plot the data using Plotly's scatter_mapbox (ensure you have a Mapbox token)
with open('/content/drive/MyDrive/Colab Notebooks/mapbox_token.txt', 'r') as file:
    mapbox_token = file.read()
px.set_mapbox_access_token(mapbox_token)

In [ ]:
import plotly.express as px
fig = px.scatter_mapbox(
    plot_data_df,
    lat='Latitude',
    lon='Longitude',
    hover_name='Name',
    hover_data={'Type': True, 'Visit Count': True},
    color='Type',  # Differentiate between professors and restaurants
    #symbol='Type',  # Use symbols for visual clarity
    mapbox_style='open-street-map',
    zoom=10,
    center={"lat": plot_data_df['Latitude'].mean(), "lon": plot_data_df['Longitude'].mean()}
)

# Customize the layout
fig.update_layout(
    legend_title_text='Location Type',
    coloraxis_colorbar={
        'title': 'Visit Counts',
        'len': 0.75,
        'thickness': 20,
        'x': 1.02,  # Move color bar to the right of the plot
        'xanchor': 'left'
    },
)

# Show the updated map
fig.show()



In [ ]:
# Load healthy food locations
healthy_food_df = pd.read_csv('/content/Healthy Food Location - Sheet1 (1).csv')  # Update path if necessary

# Assuming the healthy food CSV also has 'Place', 'Latitude', 'Longitude' columns
# Initialize visit counts for healthy food locations
healthy_visit_counts = {Place: 0 for Place in healthy_food_df['place']}

# Initialize a set to track daily visits to each healthy food location
healthy_daily_visit_tracker = {}

# Update plot data for healthy food
for _, prof_row in daily_locations.iterrows():  # Use daily_locations to ensure unique daily professor locations
    prof_location = (prof_row['latitude'], prof_row['longitude'])
    visit_date = prof_row['date']  # Get the visit date from the row

    visited_today = False  # Track if this professor's location resulted in a visit today

    for _, food_row in healthy_food_df.iterrows():
        food_location = (food_row['latitude'], food_row['longitude'])

        # Check if the professor's location matches a healthy food location
        if (round(prof_location[0], 4), round(prof_location[1], 4)) == (round(food_location[0], 4), round(food_location[1], 4)):
            # Only count one visit per day for each location
            if (food_row['place'], visit_date) not in healthy_daily_visit_tracker:
                healthy_visit_counts[food_row['place']] += 1
                healthy_daily_visit_tracker[(food_row['place'], visit_date)] = True
                visited_today = True

                # Add data to plot
                plot_data.append({
                    'Type': 'Professor',
                    'Name': 'Professor',
                    'latitude': prof_location[0],
                    'longitude': prof_location[1],
                    'Visit Count': None
                })
                plot_data.append({
                    'Type': 'Healthy Food',
                    'Name': food_row['place'],
                    'latitude': food_location[0],
                    'longitude': food_location[1],
                    'Visit Count': healthy_visit_counts[food_row['place']]
                })

            # Stop checking other healthy food locations if a visit was already recorded
            if visited_today:
                break

# Convert updated plot data to DataFrame
plot_data_df = pd.DataFrame(plot_data)

# Create the visualization
fig1 = px.scatter_mapbox(
    plot_data_df,
    lat='latitude',
    lon='longitude',
    hover_name='Name',
    hover_data={'Type': True, 'Visit Count': True},
    color='Type',  # Differentiates 'Fast Food' and 'Healthy Food'
    mapbox_style='open-street-map',
    zoom=10,
    center={"lat": plot_data_df['latitude'].mean(), "lon": plot_data_df['longitude'].mean()}
)

# Customize the layout
fig1.update_layout(
    legend_title_text='Location Type',
    coloraxis_colorbar={
        'title': 'Visit Counts',
        'len': 0.75,
        'thickness': 20,
        'x': 1.02,  # Move color bar to the right of the plot
        'xanchor': 'left'
    },
)

# Show the updated map
fig1.show()

In [ ]:
import plotly.express as px

# Sum up visit counts for fast food and healthy food locations
fast_food_total_visits = sum(visit_counts.values())
healthy_food_total_visits = sum(healthy_visit_counts.values())

# Create a DataFrame for the bar graph
bar_data = pd.DataFrame({
    'Category': ['Fast Food', 'Healthy Food'],
    'Visit Counts': [fast_food_total_visits, healthy_food_total_visits]
})

# Create the bar graph using Plotly Express
bar_fig = px.bar(
    bar_data,
    x='Category',
    y='Visit Counts',
    text='Visit Counts',
    title='Comparison of Visits: Fast Food vs. Healthy Food',
    color='Category',  # Assign different colors to the bars
    color_discrete_map={'Fast Food': 'red', 'Healthy Food': 'green'},  # Customize colors
)

# Customize the layout
bar_fig.update_traces(textposition='outside')  # Display counts outside the bars
bar_fig.update_layout(
    xaxis_title='Category',
    yaxis_title='Number of Visits',
    showlegend=False  # Hide legend since we only have two categories
)

# Show the bar graph
bar_fig.show()
